In [1]:
import pandas as pd
import numpy as np

In [2]:
# Leer archivos finales de supermercados
plazavea = pd.read_parquet('../supermercados/plazavea/plazavea.parquet')
# metro = pd.read_parquet('../supermercados/metro/metro.parquet')
# wong = pd.read_parquet('../supermercados/wong/wong.parquet')
# vivanda = pd.read_parquet('../supermercados/vivanda/vivanda.parquet')

In [3]:
# Provisional
df = plazavea

In [4]:
# Concatenar todos los dataframes en df
# df = pd.concat([plazavea, metro, wong, vivanda], ignore_index=True)

In [5]:
df.sample(n=3)

,product_name,link,descripcion_producto,price,source,categories
777,Combo Supreme - Supreme Isolate 6.6 Libras Vai...,https://www.plazavea.com.pe/combo-supreme-supr...,plazaVea,529.9,plazaVea,"[Cuidado Personal y Salud, Vitaminas y Nutrici..."
39318,Accesorio para Manguera de Alta Presión,https://www.plazavea.com.pe/accesorio-para-man...,plazaVea,84.5,plazaVea,"[Automotriz, Accesorios para Auto, Seguridad V..."
7628,"Arena para Gatos, Roedores y Aves Klin Kat 20kg",https://www.plazavea.com.pe/arena-para-gatos-r...,plazaVea,112.9,plazaVea,"[Mascotas, Salud e higiene para gatos, Arena p..."


In [6]:
df.columns

Index(['product_name', 'link', 'descripcion_producto', 'price', 'source',
       'categories'],
      dtype='object')

In [7]:
df['categories'] = df['categories'].apply(
    lambda x: ', '.join(x) if isinstance(x, (list, np.ndarray)) else str(x)
).fillna("Sin categoría")

# Utilizamos ChromaDB y Embeddings

In [8]:
!pip install chromadb sentence-transformers --quiet

In [9]:
!pip install --upgrade jupyter ipywidgets --quiet

In [10]:
!pip install tqdm --quiet

In [11]:
# !jupyter nbextension enable --py widgetsnbextension

In [12]:
from chromadb.config import Settings
from chromadb import Client
from sentence_transformers import SentenceTransformer
from tqdm.autonotebook import tqdm, trange

/home/luishuacho/python-envs/.env-3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [13]:
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer

# Configuración para guardar ChromaDB de forma permanente
client = PersistentClient(path="./chroma_db")  # Carpeta donde se guardará la base de datos

# Crear una colección
collection = client.get_or_create_collection(name="product_embeddings")

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Puedes elegir otro modelo si lo prefieras

# Inicializar variables para el contador
batch_size = 100
total_records = len(df)

# Iterar en bloques de 100 registros
for i in range(0, total_records, batch_size):
    batch = df.iloc[i:i+batch_size]  # Seleccionar un bloque de registros
    
    # Extraer y generar embeddings para product_name
    product_names = batch['product_name'].astype(str).tolist()
    product_name_embeddings = embedding_model.encode(product_names).tolist()
    
    # Extraer y generar embeddings para categories (listas convertidas a texto)
    categories = batch['categories'].apply(
        lambda x: ', '.join(x) if isinstance(x, (list, np.ndarray)) else str(x)
    ).tolist()
    categories_embeddings = embedding_model.encode(categories).tolist()
    
    # Crear metadatos para el bloque actual
    metadata = [
        {
            "source": row['source'],
            "link": row['link'],
            "price": row['price'],
            "categories": ', '.join(row['categories']) if isinstance(row['categories'], (list, np.ndarray)) else str(row['categories'])
        }
        for _, row in batch.iterrows()
    ]
    
    # Insertar datos del bloque en la colección
    collection.add(
        embeddings=categories_embeddings,  # Usar embeddings de categories para consultas primarias
        metadatas=metadata,
        documents=product_names,  # Guardar los nombres de los productos como documentos
        ids=[str(idx) for idx in range(i, min(i+batch_size, total_records))]
    )

    # Imprimir progreso
    # print(f"{min(i+batch_size, total_records)} de {total_records} registros procesados.")

print("Embeddings y metadatos guardados en ChromaDB de forma permanente.")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings y metadatos guardados en ChromaDB de forma permanente.


In [14]:
# Listar todas las colecciones disponibles
collections = client.list_collections()
print([col.name for col in collections])

['product_embeddings']


In [15]:
# Cargar la colección específica
collection = client.get_collection(name="product_embeddings")

In [16]:
# Obtener datos por ID
data_by_id = collection.get(ids=["0", "1", "2"])  # IDs específicas
print(data_by_id)

{'ids': ['0', '1', '2'], 'embeddings': None, 'documents': ['Agua Tónica BRITVIC Paquete 12un Lata 150ml', 'Agua Tónica BRTIVIC Low Calorie Lata 150ml Paquete 12un', 'Ginger Ale Beer BRITVIC Lata 150ml Paquete 12un'], 'uris': None, 'data': None, 'metadatas': [{'categories': 'Bebidas, Aguas, Agua Tónica', 'link': 'https://www.plazavea.com.pe/agua-tonica-britvic-paquete-12un-x-lata-150ml/p', 'price': 57.9, 'source': 'plazaVea'}, {'categories': 'Bebidas, Aguas, Agua Tónica', 'link': 'https://www.plazavea.com.pe/agua-tonica-brtivic-low-calorie-lata-150ml-paquete-12un/p', 'price': 57.9, 'source': 'plazaVea'}, {'categories': 'Bebidas, Gaseosas, Ginger Ale', 'link': 'https://www.plazavea.com.pe/ginger-ale-beer-britvic-lata-150ml-paquete-12un/p', 'price': 57.9, 'source': 'plazaVea'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [17]:
def search_products(query, precision=0.5, top_k=10):
    """
    Realiza una búsqueda en la base de datos ChromaDB por categorías y filtra por nombres de productos.
    
    Args:
        query (str): Texto de consulta.
        precision (float): Umbral de similitud (0.0 a 1.0) para los resultados relevantes.
        top_k (int): Número máximo de resultados a devolver en la búsqueda inicial.

    Returns:
        list: Lista de resultados relevantes con ID, nombre del producto, metadatos, y distancia.
    """
    # Generar embedding para la consulta
    query_embedding = embedding_model.encode([query]).tolist()
    
    # Primera búsqueda: categorías
    category_results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k,
        include=['documents', 'metadatas', 'distances']
    )
    
    # Filtrar resultados relevantes por precisión en categorías
    filtered_results = []
    for i, distance in enumerate(category_results['distances'][0]):
        if distance <= precision:  # Solo considerar resultados dentro del umbral de precisión
            filtered_results.append({
                "id": category_results['ids'][0][i],
                "product_name": category_results['documents'][0][i],
                "metadata": category_results['metadatas'][0][i],
                "distance": distance
            })
    
    # Si no hay resultados relevantes por categoría, retornar vacío
    if not filtered_results:
        return []
    
    # Segunda búsqueda: filtrar por nombre del producto dentro de los resultados de categorías
    refined_results = []
    for result in filtered_results:
        product_name = result["product_name"]
        product_name_embedding = embedding_model.encode([product_name]).tolist()
        similarity = embedding_model.similarity(query_embedding, product_name_embedding)
        if similarity >= precision:  # Filtrar por umbral de precisión
            result["similarity"] = similarity
            refined_results.append(result)

    return refined_results


In [18]:
# Ejemplo de búsqueda
query = "aceite 5W30 lubricante"
precision = 0.8
top_k = 10

results = search_products(query, precision=precision, top_k=top_k)

# Mostrar resultados
for result in results:
    print(result)


In [19]:
# # Crear un embedding para consulta
# query_text = "aceite para freir papas"
# query_embedding = embedding_model.encode([query_text]).tolist()
# 
# # Buscar los registros más cercanos
# results = collection.query(
#     query_embeddings=query_embedding,
#     n_results=5,  # Número de resultados más cercanos
#     include=['embeddings', 'metadatas', 'documents', 'distances']
# )
# print(results)
